In [ ]:
# https://www.youtube.com/watch?v=rIV1EseKwU4&ab_channel=AIAnytime

In [54]:
!pip install -q transformers

In [55]:
!pip install -q chromadb
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers
!pip install -q langchain
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q bitsandbytes
!pip install -q trl
!pip install -q sentence_transformers

In [56]:
!transformers-cli env


Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.39.2
- Platform: Windows-10-10.0.22621-SP0
- Python version: 3.11.8
- Huggingface_hub version: 0.20.3
- Safetensors version: 0.4.2
- Accelerate version: 0.28.0
- Accelerate config: 	not found
- PyTorch version (GPU?): 2.2.2+cu121 (True)
- Tensorflow version (GPU?): not installed (NA)
- Flax version (CPU?/GPU?/TPU?): not installed (NA)
- Jax version: not installed
- JaxLib version: not installed
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>



In [57]:
import os
import torch
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [58]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.2'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [59]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "bfloat16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [60]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [61]:
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

RuntimeError: No GPU found. A GPU is needed for quantization.

In [52]:
#################################################################
# Base inference
#################################################################

prompt = """En que consiste el Artículo 5 de la ley 11/2022 del BOE"""

# CUDA: Para programar directamente la GPU
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))

AssertionError: Torch not compiled with CUDA enabled

## LangChain

In [3]:
import pypdf
from langchain.document_loaders import DirectoryLoader, PyPDFLoader, PDFMinerLoader
from pypdf import PdfReader

CHROMA_PATH = "chroma"
DATA_PATH = "data"

def get_pdf_text():
    loader = PyPDFLoader("data/BOE-A-2022-10757.pdf")
    pages = loader.load_and_split()
    
    return pages

In [ ]:
documents = get_pdf_text()
print(documents[0])

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2500,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True
    )
    
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    # Cogemos un chunk aleatorio
    document = chunks[10]
    print(document.page_content) # Printeamos su contenido
    print(document.metadata) # Y su metadata (Fichero al que pertenece y donde empieza 'start_index')
    
    return chunks

In [6]:
chunks = split_text(documents)

Split 247 documents into 400 chunks.
PREÁMBULO
I
La Ley 9/2014, de 9 de mayo, General de Telecomunicaciones, introdujo reformas 
estructurales en el régimen jurídico de las telecomunicaciones dirigidas a facilitar el 
despliegue de redes y la prestación de servicios por parte de los operadores. Dicha ley 
estableció las bases para asegurar que la extensión de las redes de nueva generación 
se llevase a cabo conforme a los principios de fomento de la inversión e impulso de la 
competencia, garantizando un marco regulatorio claro y estable, que ha proporcionado 
seguridad jurídica y eliminado barreras que dificultaban el despliegue de redes. Ello ha 
permitido a los operadores ofrecer a los usuarios servicios innovadores, de mayor 
calidad y cobertura, a precios competitivos y con mejores condiciones, contribuyendo de 
este modo a potenciar la competitividad y la productividad de la economía española en 
su conjunto.
En la actualidad, las redes alcanzan en nuestro país una cobertura del 

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
import shutil
import chromadb

def save_to_chroma(chunks):
    
    # Clear out the database first if already exists.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
    
    #embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
    embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
    
    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, 
        embeddings, 
        persist_directory=CHROMA_PATH
    )
    
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

    return embeddings

In [14]:
embeddings = save_to_chroma(chunks)

PermissionError: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'chroma\\7c69b794-265c-4c48-a486-8f2f5a3ca938\\data_level0.bin'

In [38]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

def load_model():
    text_generation_pipeline = transformers.pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        repetition_penalty=1.1,
        return_full_text=True,
        max_new_tokens=300
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf

In [ ]:
llm = load_model()

In [40]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain

def query_data2(embeddings, llm):
    
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
    retriever = db.as_retriever()

    prompt_template = """
    ### [INST] Instrucción: Eres un experto en jurisprudencia y todo lo relacionado con jurisdiccion, responde en español a la pregunta según tus conocimientos de jurisdiccion y el siguiente contexto:
    
    {context}
    
    ### PREGUNTA:
    {question} (responde en castellano) [/INST]
     """
    
    promptTemplate = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template
    )
    
    llm_chain = LLMChain(llm=llm, prompt=promptTemplate)

    # RunnablePassthrough para pasar la query al siguiente step en la chain
    rag_chain = ( 
     {"context": retriever, "question": RunnablePassthrough()}
        | llm_chain
    )

    return rag_chain

In [ ]:
rag_chain = query_data2()
answer = rag_chain.invoke(query)

In [42]:
answer['context']
answer['text']

In [ ]:
    prompt_template = """
    ### [INST] 
    Instruction: Answer the question based on your 
    fantasy football knowledge. Here is context to help:

    {context}

    ### QUESTION:
    {question} 

    [/INST]
     """